In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# abc_0051_00515110_43a0e83aaca82954cacdbe14_005

In [3]:
from copy import deepcopy

import k3d
import yaml
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm
import json
import pymesh

import torch

import trimesh.transformations as tt
import trimesh
from io import BytesIO

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-y4ccgc13 because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [197]:
import os
os.environ['OMP_NUM_THREADS'] = '38'

In [5]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768,
                      cmap=k3d.colormaps.matplotlib_color_maps.coolwarm_r):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color, flat_shading=False)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 1.0

            colors = k3d.helpers.map_colors(
                samples_distances, cmap, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [6]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [46]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.camera_pose_manager import POSE_MANAGER_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_depth_maps
from sharpf.data.imaging import IMAGING_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.utils.abc_utils.abc import feature_utils
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.plotting import display_depth_sharpness, illustrate_camera
from sharpf.utils.camera_utils.camera_pose import camera_to_display
from sharpf.utils.abc_utils.mesh.indexing import reindex_zerobased, compute_relative_indexes
import sharpf.data.data_smells as smells

from sharpf.utils.camera_utils.spherical_spiral_sampling import spherical_spiral_sampling

In [47]:
!ls /data

3D_normals_dataset_generation.ipynb  realworld_sharpf_scans_raw
_SKOLTECH.rar			     scannet
abc				     sharp_features_data
colmap_test			     shm_stbasil
mesh_denoising			     toy
realworld_sharpf_scans


In [173]:
with ABCChunk(['/data/abc/abc_0051_obj_v00.7z', '/data/abc/abc_0051_feat_v00.7z']) as data_holder:
    item = data_holder.get('00510073_951e25d2ded40f22b598f84e_000')
item.item_id

'00510073_951e25d2ded40f22b598f84e_000'

In [174]:
mesh, _, _ = trimesh_load(item.obj)
features = yaml.load(item.feat, Loader=yaml.Loader)

In [52]:
pymesh.detect_self_intersection(mesh)

array([], shape=(0, 2), dtype=int32)

In [184]:
config = {
  "shape_fabrication_extent": 10.0,
  "short_curve_quantile": 0.25,
  "base_n_points_per_short_curve": 8,
  "base_resolution_3d": 0.125,
  "camera_pose": {
    "type": "sphere_spiral_to_origin",
    "n_images": 0,
    "layer_radius": 0.1,
    "resolution": 0.02,
    "n_initial_samples": 1001,
    "min_arc_length": 0
#     "type": "composite",
#     "sequences": [
#         {
#             "type": "sphere_spiral_to_origin",
#             "n_images": 0,
#             "layer_radius": 0.1,
#             "resolution": 0.01,
#             "n_initial_samples": 10001,
#             "min_arc_length": 0.1
#         }
#       {
#         "type": "sphere_to_origin",
#         "n_images": 2
#       },
#       {
#         "type": "xy_translation",
#         "n_images": 8*8
#       },
#       {
#         "type": "z_rotation",
#         "n_images": 5
#       }
#     ]
  },
  "imaging": {
    "type": "raycasting",
    "projection": "ortho",
    "resolution_image": 512,
    "resolution_3d": 0.02,
    "fov": [115, 85, 80],
    "validate_image": True
  },
  "noise": {
    "type": "z_direction",
    "scale": 0.0
  },
  "annotation": {
    "type": "surface_based_aabb",
    "distance_upper_bound": 1.0,
    "distance_computation_method": "geom",
  },
  "smell_coarse_surfaces_by_num_edges": {
    "num_edges_threshold": 8
  },
  "smell_coarse_surfaces_by_angles": {
    "max_angle_threshold_degrees": 10.0
  },
  "smell_deviating_resolution": {
    "resolution_3d": 0.02,
    "resolution_deviation_tolerance": 0.01
  },
  "smell_sharpness_discontinuities": { },
  "smell_bad_face_sampling": {
    "min_points_per_face": 0.02,
    "max_points_per_face": 20.0
  },
  "smell_raycasting_background": { },
  "smell_mesh_self_intersections": { },
  "smell_depth_discontinuity": {
    "depth_discontinuity_threshold": 0.5
  }
}

In [185]:
shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)

short_curve_quantile = config.get('short_curve_quantile', 0.05)

pose_manager = load_func_from_config(POSE_MANAGER_BY_TYPE, config['camera_pose'])
imaging = load_func_from_config(IMAGING_BY_TYPE, config['imaging'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])

{'composite': <class 'sharpf.data.camera_pose_manager.CompositePoseManager'>, 'sphere_to_origin': <class 'sharpf.data.camera_pose_manager.SphereOrientedToWorldOrigin'>, 'sphere_spiral_to_origin': <class 'sharpf.data.camera_pose_manager.SphericalSpiralOrientedToWorldOrigin'>, 'z_rotation': <class 'sharpf.data.camera_pose_manager.ZRotationInCameraFrame'>, 'xy_translation': <class 'sharpf.data.camera_pose_manager.XYTranslationInCameraFrame'>} {'type': 'sphere_spiral_to_origin', 'n_images': 0, 'layer_radius': 0.1, 'resolution': 0.02, 'n_initial_samples': 1001, 'min_arc_length': 0}
{'raycasting': <class 'sharpf.data.imaging.RaycastingImaging'>} {'type': 'raycasting', 'projection': 'ortho', 'resolution_image': 512, 'resolution_3d': 0.02, 'fov': [115, 85, 80], 'validate_image': True}
{'no_noise': <class 'sharpf.data.noisers.NoNoise'>, 'isotropic_gaussian': <class 'sharpf.data.noisers.IsotropicGaussianNoise'>, 'normals_gaussian': <class 'sharpf.data.noisers.NormalsGaussianNoise'>, 'z_direction

In [186]:
smell_coarse_surfaces_by_num_edges = smells.SmellCoarseSurfacesByNumEdges.from_config(config['smell_coarse_surfaces_by_num_edges'])
smell_coarse_surfaces_by_angles = smells.SmellCoarseSurfacesByAngles.from_config(config['smell_coarse_surfaces_by_angles'])
smell_deviating_resolution = smells.SmellDeviatingResolution.from_config(config['smell_deviating_resolution'])
smell_sharpness_discontinuities = smells.SmellSharpnessDiscontinuities.from_config(config['smell_sharpness_discontinuities'])
smell_bad_face_sampling = smells.SmellBadFaceSampling.from_config(config['smell_bad_face_sampling'])
smell_raycasting_background = smells.SmellRaycastingBackground.from_config(config['smell_raycasting_background'])
smell_depth_discontinuity = smells.SmellDepthDiscontinuity.from_config(config['smell_depth_discontinuity'])
smell_mesh_self_intersections = smells.SmellMeshSelfIntersections.from_config(config['smell_mesh_self_intersections'])


In [163]:
def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = feature_utils.get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    scale = least_len_mm / least_len
    mesh = mesh.apply_scale(scale)

    return mesh, scale


In [132]:
mesh, mesh_scale = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                              short_curve_quantile=short_curve_quantile,
                              n_points_per_short_curve=base_n_points_per_short_curve)

mesh = mesh.apply_translation(-mesh.vertices.mean(axis=0))

In [175]:
mesh = mesh.apply_scale(shape_fabrication_extent / np.max(mesh.bounding_box.extents))
mesh = mesh.apply_scale(mesh_scale)
mesh = mesh.apply_translation(-mesh.vertices.mean(axis=0))

In [177]:
np.max(mesh.bounding_box.extents)

6.040656477057844

In [134]:
_ = trimesh.exchange.export.export_mesh(
    mesh,
    '/logs/abc_0051_00510073_951e25d2ded40f22b598f84e_000.obj',
    'obj')

In [240]:
from sharpf.data.imaging import RaycastingImaging
import sharpf.data.datasets.sharpf_io as sharpf_io
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes
from typing import List, Mapping
from functools import partial
from sharpf.utils.camera_utils.camera_pose import CameraPose, camera_to_display


resolution_3d = 0.02

d = '/logs/paper_extra/spherical_sphiral/'
gt_filename = os.path.join(d, 'abc_0051_64_65.hdf5')

ground_truth_dataset = Hdf5File(
    gt_filename,
    io=sharpf_io.WholeDepthMapIO,
    preload=PreloadTypes.LAZY,
    labels='*')
gt_dataset = [view for view in ground_truth_dataset]
# depth images captured from a variety of views around the 3D shape
gt_images = [view['image'] for view in gt_dataset]
# ground-truth distances (multi-view consistent for the global 3D shape)
gt_distances = [view.get('distances', np.ones_like(view['image'])) for view in gt_dataset]
# extrinsic camera matrixes describing the 3D camera poses used to capture depth images
gt_extrinsics = [view['camera_pose'] for view in gt_dataset]
# intrinsic camera parameters describing how to compute image from points and vice versa
gt_intrinsics = [dict(resolution_image=gt_images[0].shape[::-1], resolution_3d=resolution_3d, projection=None, validate_image=None) for view in
                 gt_dataset]

def get_view(
        images: List[np.array],
        distances: List[np.array],
        extrinsics: List[np.array],
        intrinsics_dict: List[Mapping],
        i):
    """A helper function to conveniently prepare view information."""
    image_i = images[i]  # [h, w]
    distances_image_i = distances[i]  # [h, w]
    # Kill background for nicer visuals
    distances_i = np.zeros_like(distances_image_i)
    distances_i[np.nonzero(image_i)] = distances_image_i[np.nonzero(image_i)]

    pose_i = CameraPose(extrinsics[i])
    imaging_i = RaycastingImaging(**intrinsics_dict[i])
    points_i = pose_i.camera_to_world(imaging_i.image_to_points(image_i))

    return image_i, distances_i, points_i, pose_i, imaging_i


get_view_local = partial(get_view, gt_images, gt_distances, gt_extrinsics, gt_intrinsics)


# view_i = get_view_local(0)
# view_j = get_view_local(1)

In [243]:
for i in tqdm(range(len(gt_dataset))):
    image, distances, points, pose, imaging = get_view_local(i)

    distances, directions, has_sharp = annotator.annotate(
        mesh, features, points)

    ray_indexes = np.where(image.ravel() != 0)[0]
    distances_image = imaging.points_to_image(distances.reshape(-1, 1), ray_indexes, assign_channels=[0])
    directions_image = imaging.points_to_image(directions, ray_indexes, assign_channels=[0, 1, 2])
    
    gt_dataset[i]['distances'] = distances_image
    gt_dataset[i]['directions'] = directions_image
    
    np.save(
        os.path.join(d, 'npy', '{0:04d}__distances.npy'.format(i)),
        distances_image)
    np.save(
        os.path.join(d, 'npy', '{0:04d}__directions.npy'.format(i)),
        directions_image)

100%|██████████| 1023/1023 [28:35:38<00:00, 100.62s/it] 


In [244]:
from sharpf.data.datasets.sharpf_io import save_depth_maps
gt_filename_out = os.path.join(d, 'abc_0051_64_65__whole_annotation.hdf5')
save_depth_maps(gt_dataset, gt_filename_out)

In [233]:
image, distances, points, pose, imaging = get_view_local(0)

In [235]:
display_sharpness(
    mesh, plot_meshvert=False, mesh_color=0x0D0887,
    sharp_vert=None, sharpvert_color=0xF0C821,
    samples=points, samples_psize=0.02, 
    samples_distances=distances[])

IndexError: index 49941 is out of bounds for axis 0 with size 512

In [149]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm


class MplColorHelper:

    def __init__(self, cmap_name, start_val, stop_val):
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        return self.scalarMap.to_rgba(val)


    
def export_points_to_npy(
    filename, 
    points, 
    distances=None, 
    pose=None, 
    radius=0.02,
    max_distance=1.0):
    
    tol = 1e-3
    if distances is None:
        if pose is not None:
            distances = np.linalg.norm(pose.frame_origin - points, axis=1)        
            helper = MplColorHelper(
                'viridis',
                np.min(distances) - tol,
                np.max(distances) + tol)
        else:
            raise ValueError()
    else:
        helper = MplColorHelper(
            'plasma_r',
            -tol,
            max_distance + tol)

    iterable = zip(points, distances)
    colors_rgb = []
    for point, distance in iterable:
        rgba = helper.get_rgb(distance)
        colors_rgb.append(rgba[:3])
        
    np.save(filename, np.hstack((points, colors_rgb)))
    print('saved {}'.format(filename))

In [151]:
export_points_to_npy(
    '/logs/abc_0051_00510073_951e25d2ded40f22b598f84e_000__points_202.npy',
    view_i[2],
    distances=view_i[1][view_i[1]!=0]
)

export_points_to_npy(
    '/logs/abc_0051_00510073_951e25d2ded40f22b598f84e_000__points_204.npy',
    view_j[2],
    distances=view_j[1][view_j[1]!=0]
)

saved /logs/abc_0051_00510073_951e25d2ded40f22b598f84e_000__points_202.npy
saved /logs/abc_0051_00510073_951e25d2ded40f22b598f84e_000__points_204.npy


In [118]:
camera_origins = spherical_spiral_sampling(
    np.max(mesh.bounding_box.extents),
    layer_radius=0.1,
    resolution=0.01,
    n_initial_samples=10001,
    min_arc_length=0.1
)

In [117]:
len(camera_origins)

10000

In [72]:
# sharp_vert_indexes = np.concatenate(
#     [c['vert_indices'] for c in features['curves'] if c['sharp']]
# )
# sharp_vert = mesh.vertices[sharp_vert_indexes]

display_sharpness(
    mesh, plot_meshvert=False, mesh_color=0x0D0887,
    sharp_vert=camera_origins, sharpvert_color=0xF0C821,
#     sharp_curves=[f['vert_indices'] for f in features['curves'] if f['sharp']], 
    sharpcurve_color=0xF0C821, sharpcurve_width=0.05,
)

Output()

In [119]:
pose_manager.prepare(mesh)

In [80]:
# plot = k3d.plot(grid_visible=False, height=1024)
# for idx, pose in tqdm(enumerate(pose_manager.camera_poses)):
#     plot += illustrate_camera(pose, w=0.05, l=0.2)
# # plot += k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb, flat_shading=False)
# plot.display()

In [120]:
def get_image_from_pose(pose, mesh, features, imaging, annotator):
    # extract neighbourhood
    try:
        image, points, normals, mesh_face_indexes = \
            imaging.get_image_from_pose(mesh, pose, return_hit_face_indexes=True)
    except DataGenerationException as e:
#         eprint_t(str(e))
        return np.zeros(imaging.resolution_image), np.zeros(imaging.resolution_image), None, None

#     nbhood, mesh_vertex_indexes, mesh_face_indexes = \
#                 feature_utils.submesh_from_hit_surfaces(mesh, features, mesh_face_indexes)

#     # create annotations: condition the features onto the nbhood
#     nbhood_features = feature_utils.compute_features_nbhood(
#         mesh, features, mesh_face_indexes, mesh_vertex_indexes=mesh_vertex_indexes,
#         deduce_verts_from_faces=False)

#     # remove vertices lying on the boundary (sharp edges found in 1 face only)
# #     nbhood_features = feature_utils.remove_boundary_features(nbhood, nbhood_features, how='edges')

#     noisy_points = noiser.make_noise(
#         pose.world_to_camera(points),
#         normals,
#         z_direction=np.array([0., 0., -1.]))

#         # compute the TSharpDF
#     try:
#         distances, directions, has_sharp = annotator.annotate(
#             nbhood, nbhood_features, pose.camera_to_world(noisy_points))
#     except DataGenerationException as e:
#         eprint_t(str(e))

#     # convert everything to images
#     ray_indexes = np.where(image.ravel() != 0)[0]
#     noisy_image = imaging.points_to_image(noisy_points, ray_indexes)
#     normals = imaging.points_to_image(normals, ray_indexes, assign_channels=[0, 1, 2])
#     distances = imaging.points_to_image(distances.reshape(-1, 1), ray_indexes, assign_channels=[0])
#     directions = imaging.points_to_image(directions, ray_indexes, assign_channels=[0, 1, 2])
        
    distances = None
    nbhood = None
    nbhood_features = None
    return image, distances, nbhood, nbhood_features



all_images, all_distances, all_patches, all_features, all_poses = [], [], [], [], []
pose_manager.prepare(mesh)
for pose in tqdm(pose_manager.camera_poses):
    image, distances, patch, patch_features = get_image_from_pose(pose, mesh, features, imaging, annotator)
    all_images.append(image)
    all_distances.append(distances)
    all_patches.append(patch)
    all_features.append(patch_features)
    all_poses.append(pose)

100%|██████████| 1000/1000 [04:18<00:00,  3.88it/s]


In [121]:
plt.ioff()

for i, image in tqdm(enumerate(all_images)):
    _ = display_depth_sharpness(
        depth_images=[
            camera_to_display(image)],
    #     sharpness_images=[
    #         camera_to_display(distances) 
    #         for distances in all_distances[pose_idx]],
        ncols=1,
        max_sharpness=1.05)
    _ = plt.savefig('/logs/images_for_video/{0:04d}.png'.format(i + 1))
    plt.close()

1000it [01:49,  9.17it/s]


In [ ]:
len(all_images)

In [ ]:
all_images, all_distances = np.array(all_images), np.array(all_distances)

In [ ]:
n_directions = 2
n_offsets_y, n_offsets_x = 8, 8
n_offsets = n_offsets_y * n_offsets_x

In [ ]:
all_images = all_images.reshape((n_directions, n_offsets_y, n_offsets_x, 64, 64))
all_images = np.rollaxis(all_images, 1, 3).reshape((2, n_offsets, 64, 64))

all_distances = all_distances.reshape((n_directions, n_offsets_y, n_offsets_x, 64, 64))
all_distances = np.rollaxis(all_distances, 1, 3).reshape((2, n_offsets, 64, 64))

In [ ]:
len(all_images)

In [ ]:
pose_idx = 0

display_depth_sharpness(
    depth_images=[
        camera_to_display(image) 
        for image in all_images[pose_idx]],
#     sharpness_images=[
#         camera_to_display(distances) 
#         for distances in all_distances[pose_idx]],
    ncols=8,
    axes_size=(4, 4),
    layout_hpad=8,
    max_sharpness=1.05,
    frame_visible=True,
)

In [ ]:
pose_idx = 0

# row_idx, col_idx = 2, 3
# row_idx, col_idx = 4, 3
# row_idx, col_idx = 4, 6
row_idx, col_idx = 3, 5
item_idx = row_idx * n_offsets_x + col_idx
print(item_idx, col_idx * n_offsets_x + row_idx)

display_depth_sharpness(
    depth_images=[
        camera_to_display(image) 
        for image in all_images[pose_idx][[item_idx]]],
    sharpness_images=[
        camera_to_display(distances) 
        for distances in all_distances[pose_idx][[item_idx]]],
    ncols=1,
    axes_size=(4, 4),
#     layout_pad=1,
    max_sharpness=1.05,
    frame_visible=True,
)

In [ ]:
# pose_manager.prepare(mesh)
pose = all_poses[43]
# plot = k3d.plot(grid_visible=False, height=1024)
# for idx, pose in tqdm(enumerate(all_poses[:item_idx])):
#     plot += illustrate_camera(pose, w=0.1, l=0.75)
# plot += k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb, flat_shading=False)
# plot.display()

display_sharpness(
    mesh, plot_meshvert=False, 
    samples=pose.camera_to_world(
        imaging.image_to_points(
            all_images[pose_idx][item_idx])),
    samples_psize=0.1,
#     sharp_vert=sharp_vert, sharpvert_color=0xF0C821,
#     sharp_curves=[f['vert_indices'] for f in features['curves'] if f['sharp']], 
#     sharpcurve_color=0xF0C821, sharpcurve_width=0.05,
)


In [ ]:
def export_points_to_npy(
    filename, 
    points, 
    distances=None, 
    pose=None, 
    radius=0.02,
    max_distance=1.0):
    
    tol = 1e-3
    if distances is None:
        if pose is not None:
            distances = np.linalg.norm(pose.frame_origin - points, axis=1)        
            helper = MplColorHelper(
                'viridis',
                np.min(distances) - tol,
                np.max(distances) + tol)
        else:
            raise ValueError()
    else:
        helper = MplColorHelper(
            'plasma_r',
            -tol,
            max_distance + tol)

    iterable = zip(points, distances)
    colors_rgb = []
    for point, distance in iterable:
        rgba = helper.get_rgb(distance)
        colors_rgb.append(rgba[:3])
        
#         point_mesh = trimesh.creation.icosphere(
#             subdivisions=1,
#             radius=radius,
#             color=rgba[:3])
#         point_mesh.vertices += point
#         full_mesh.append(point_mesh)
#     full_mesh = trimesh.util.concatenate(full_mesh)
#     _ = trimesh.exchange.export.export_mesh(
#         full_meshes,
#         'filename',
#         'obj')

    np.save(filename, np.hstack((points, colors_rgb)))
    print('saved {}'.format(filename))

In [ ]:
export_points_to_npy('/logs/data_generation__image_3.npy', pose.camera_to_world(
        imaging.image_to_points(
            all_images[pose_idx][item_idx])), pose=pose)

In [ ]:
display_sharpness(all_patches[52])

In [ ]:
def get_edges(mesh, features):
    curve_segments = []
    for curve_id, curve in enumerate(features['curves']):
        if curve['sharp']:
            # (n, 2) mask of boolean variables where each variable stores
            # True if the corresponding vertex idx is in curve
            is_edgevert_in_curve = np.isin(mesh.edges_unique, curve['vert_indices'])
            is_edge_in_curve = np.all(is_edgevert_in_curve, axis=1)
            edge_in_curve_indexes = np.where(is_edge_in_curve)[0]
            curve_edges = mesh.edges_unique[edge_in_curve_indexes]  # n_curve, 2
            curve_segment_xyz = mesh.vertices[curve_edges]  # n_curve, 2, 3
            curve_segments.append((curve_id, curve['type'], np.ravel(curve_segment_xyz)))

    return curve_segments


def export_features_to_obj(output_filename, curve_segments):
    with open(output_filename, 'w') as fobj:
        vertices = ''
        indices = ''
        max_vi = 1
        for curve_id, curve_type, xyz_xyz in curve_segments:
            xyz_xyz = xyz_xyz.reshape((-1, 2, 3))
            for v1, v2 in xyz_xyz:
                vertices += 'v {v1}\nv {v2}\n'.format(
                    v1=' '.join([str(coord) for coord in v1]),
                    v2=' '.join([str(coord) for coord in v2])
                )
            for edge_idx, edge in enumerate(xyz_xyz):
                indices += 'l {i1} {i2}\n'.format(
                    i1=str(max_vi + edge_idx * 2),
                    i2=str(max_vi + edge_idx * 2 + 1),
                )
            max_vi += 2 * len(xyz_xyz)
        fobj.write(vertices + '\n' + indices)

In [ ]:
for idx, data_idx in enumerate([26, 28, 43, 52]):
    curve_segments = get_edges(all_patches[data_idx], all_features[data_idx])
    trimesh.exchange.export.export_mesh(
        all_patches[data_idx], 
        '/logs/figures_for_paper/2-data-pipeline-images-{}-mesh.obj'.format(idx), 
        'obj')
    export_features_to_obj(
        '/logs/figures_for_paper/2-data-pipeline-images-{}-features.obj'.format(idx), 
        curve_segments)

In [ ]:
# generate camera poses
pose_manager.prepare(mesh)

pose_manager_iter = iter(pose_manager)

In [ ]:
# camera_pose = next(pose_manager_iter)

# # extract neighbourhood
# try:
#     image, points, normals, mesh_face_indexes = \
#         imaging.get_image_from_pose(mesh, camera_pose, return_hit_face_indexes=True)
# except DataGenerationException as e:
#     eprint_t(str(e))


plot = k3d.plot(grid_visible=False, height=1024)
for idx, pose in tqdm(enumerate(pose_manager)):
    if idx % 6 == 0:
        plot += illustrate_camera(pose, w=0.1, l=0.75)
    if idx == 63:
        break
plot += k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb, flat_shading=False)
plot.display()

In [ ]:
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE

config_points = {
  "shape_fabrication_extent": 10.0,
  "short_curve_quantile": 0.25,
  "base_n_points_per_short_curve": 8,
  "base_resolution_3d": 0.125,
  "neighbourhood": {
    "type": "random_euclidean_sphere",
    "max_patches_per_mesh": 128,
    "n_vertices": None,
    "centroid": None,
    "centroid_mode": "poisson_disk",
    "radius_base": 10.0,
    "radius_delta": 0.0,
    "geodesic_patches": True,
    "radius_scale_mode": "no_scale"
  },
  "sampling": {
    "type": "poisson_disk",
    "n_points": 4096,
    "resolution_3d": 0.05,
    "make_n_points": "crop_center"
  },
  "noise": {
    "type": "many_noisers",
    "subtype": "isotropic_gaussian",
    "scale": [0.0]
  },
  "annotation": {
    "type": "surface_based_aabb",
    "distance_upper_bound": 1.0
  },
  "smell_coarse_surfaces_by_num_edges": {
    "num_edges_threshold": 8
  },
  "smell_coarse_surfaces_by_angles": {
    "max_angle_threshold_degrees": 10.0
  },
  "smell_deviating_resolution": {
    "resolution_3d": 0.125,
    "resolution_deviation_tolerance": 0.0625
  },
  "smell_sharpness_discontinuities": {
  },
  "smell_bad_face_sampling": {
    "min_points_per_face": 0.02,
    "max_points_per_face": 20.0
  }
}

shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)

short_curve_quantile = config.get('short_curve_quantile', 0.05)

nbhood_extractor = load_func_from_config(NBHOOD_BY_TYPE, config_points['neighbourhood'])
sampler = load_func_from_config(SAMPLER_BY_TYPE, config_points['sampling'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config_points['annotation'])

# Specific to this script only: override radius of neighbourhood extractor
# to reflect actual point cloud resolution:
# we extract spheres of radius r, such that area of a (plane) disk with radius r
# is equal to the total area of 3d points (as if we scanned a plane wall)
nbhood_extractor.radius_base = np.sqrt(sampler.n_points) * 0.5 * sampler.resolution_3d / 1.5


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm


class MplColorHelper:

    def __init__(self, cmap_name, start_val, stop_val):
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        return self.scalarMap.to_rgba(val)


In [ ]:



np.random.seed(5371)
nbhood_extractor.index(mesh)

LARGEST_PROCESSABLE_MESH_VERTICES = 20000

all_points, all_distances_points, all_patches_points, all_features_points = [], [], [], []
for current_patch_idx in tqdm(range(len(nbhood_extractor.centroids_cache))):

    # extract neighbourhood
    try:
        nbhood, mesh_vertex_indexes, mesh_face_indexes, scaler = nbhood_extractor.get_nbhood()
        if len(nbhood.vertices) > LARGEST_PROCESSABLE_MESH_VERTICES:
            raise DataGenerationException('Too large number of vertices in crop: {}'.format(len(nbhood.vertices)))
    except DataGenerationException as e:
        eprint_t(str(e))
    centroid = nbhood_extractor.centroid

    # sample the neighbourhood to form a point patch
    try:
        points, normals = sampler.sample(nbhood, centroid=None)
    except Exception as e:
        eprint(str(e))


    # create annotations: condition the features onto the nbhood
    nbhood_features = feature_utils.compute_features_nbhood(mesh, features, mesh_face_indexes, mesh_vertex_indexes=mesh_vertex_indexes)

    # remove vertices lying on the boundary (sharp edges found in 1 face only)
    nbhood_features = feature_utils.remove_boundary_features(nbhood, nbhood_features, how='edges')

    try:
        distances, directions, has_sharp = annotator.annotate(nbhood, nbhood_features, points)
    except DataGenerationException as e:
        eprint_t(str(e))

    
    all_points.append(points)
    all_distances_points.append(distances)
    all_patches_points.append(nbhood)
    all_features_points.append(nbhood_features)
        

In [ ]:
full_meshes = []

for points, distances in zip(all_points[4:5], all_distances_points[4:5]):

    distances_to_camera = np.linalg.norm(pose.frame_origin - points, axis=1)
    tol = 1e-3
#     helper = MplColorHelper(
#         'viridis',
#         np.min(distances_to_camera) - tol,
#         np.max(distances_to_camera) + tol)
    helper = MplColorHelper(
        'plasma_r',
        - tol,
        1. + tol)

    iterable = zip(points, distances)
    full_mesh = []
    for point, distance in iterable:
        rgba = helper.get_rgb(distance)
        point_mesh = trimesh.creation.icosphere(
            subdivisions=1,
            radius=0.02,
            color=rgba[:3])
        point_mesh.vertices += point
        full_mesh.append(point_mesh)

    full_mesh = trimesh.util.concatenate(full_mesh)

    full_meshes.append(full_mesh)


In [ ]:
full_meshes = trimesh.util.concatenate(full_meshes)

_ = trimesh.exchange.export.export_mesh(
    full_meshes,
    '/logs/figures_for_paper/2-data-pipeline-points-3-distances.obj',
    'obj')

In [ ]:
for idx, data_idx in enumerate([8, 15, 4]):
    curve_segments = get_edges(all_patches_points[data_idx], all_features_points[data_idx])
    trimesh.exchange.export.export_mesh(
        all_patches_points[data_idx], 
        '/logs/figures_for_paper/2-data-pipeline-points-{}-mesh.obj'.format(idx + 1), 
        'obj')
    export_features_to_obj(
        '/logs/figures_for_paper/2-data-pipeline-points-{}-features.obj'.format(idx + 1), 
        curve_segments)

In [ ]:
plot = k3d.plot(grid_visible=False, height=1024)

idx = 15
points, distances = all_points[idx], all_distances_points[idx]

d = np.linalg.norm(pose.frame_origin - points, axis=1)
colors = k3d.helpers.map_colors(
    d, 
    k3d.colormaps.matplotlib_color_maps.viridis, [np.min(d), np.max(d)]
).astype(np.uint32)

plot += k3d.points(
    points, 
    colors=colors,
    shader='flat',
    point_size=0.02,)

plot.display()

In [ ]:
plot = k3d.plot(grid_visible=False, height=1024)
for full_mesh in full_meshes:
    plot += k3d.mesh(full_mesh.vertices, full_mesh.faces, color=0xbbbbbb, flat_shading=False)
plot.display()

In [ ]:
nbhood, mesh_vertex_indexes, mesh_face_indexes = \
            feature_utils.submesh_from_hit_surfaces(mesh, features, mesh_face_indexes)

# create annotations: condition the features onto the nbhood
nbhood_features = feature_utils.compute_features_nbhood(
    mesh, features, mesh_face_indexes, mesh_vertex_indexes=mesh_vertex_indexes,
    deduce_verts_from_faces=False)

# remove vertices lying on the boundary (sharp edges found in 1 face only)
nbhood_features = feature_utils.remove_boundary_features(nbhood, nbhood_features, how='edges')

In [ ]:
nbhood_sharp_vert_indexes = np.concatenate(
    [c['vert_indices'] for c in nbhood_features['curves'] if c['sharp']]
)
nbhood_sharp_vert = nbhood.vertices[nbhood_sharp_vert_indexes]


display_sharpness(
    nbhood, plot_meshvert=True, meshvert_psize=imaging.resolution_3d / 2,
#     sharp_vert=camera_pose.camera_to_world(imaging.rays_origins), sharpvert_psize=0.5,
    sharp_vert=nbhood_sharp_vert, sharpvert_psize=0.2,
    samples=camera_pose.camera_to_world(imaging.image_to_points(image)), samples_psize=0.1,
    directions=None
)

In [ ]:
noisy_points = noiser.make_noise(
    camera_pose.world_to_camera(points),
    normals,
    z_direction=np.array([0., 0., -1.]))

In [ ]:
import igl

In [ ]:
_, point_face_indexes, _ = \
            igl.point_mesh_squared_distance(noisy_points, nbhood.vertices, nbhood.faces)

In [ ]:
from sharpf.utils.abc_utils.abc.feature_utils import get_adjacent_features_by_bfs_with_depth1, build_surface_patch_graph

In [ ]:
adjacent_sharp_features, adjacent_surfaces = build_surface_patch_graph(nbhood, nbhood_features)

In [ ]:
adjacent_sharp_features

In [ ]:
adjacent_surfaces

In [ ]:
from collections import defaultdict
def get_adjacent_features_by_bfs_with_depth1(
        surface_idx,
        adjacent_sharp_features,
        adjacent_surfaces,
        always_check_adjacent_surfaces=False
):
    """If adjacent sharp curves exist, return one of them.
    If not, return ones adjacent to adjacent surfaces. """

    adjacent_sharp_indexes = deepcopy(adjacent_sharp_features[surface_idx])
    if always_check_adjacent_surfaces or len(adjacent_sharp_indexes) == 0:

        for adjacent_surface_idx in adjacent_surfaces[surface_idx]:
            adjacent_surface_adjacent_sharp_features = \
                {adjacent_surface_idx: adjacent_sharp_features[adjacent_surface_idx]}

            adjacent_surface_adjacent_sharp_indexes = \
                get_adjacent_features_by_bfs_with_depth1(
                    adjacent_surface_idx, adjacent_surface_adjacent_sharp_features,
                    defaultdict(list))

            adjacent_sharp_indexes.extend(adjacent_surface_adjacent_sharp_indexes)

    return adjacent_sharp_indexes


In [ ]:
surface_idx = 2

surface = nbhood_features['surfaces'][surface_idx]

# constrain distance computation to certain sharp features only
adjacent_sharp_indexes = get_adjacent_features_by_bfs_with_depth1(
    surface_idx, adjacent_sharp_features, adjacent_surfaces, always_check_adjacent_surfaces=True)

nbhood_sharp_vert_indexes = np.concatenate(
    [nbhood_features['curves'][curve_idx]['vert_indices'] 
     for curve_idx in adjacent_sharp_indexes
     if nbhood_features['curves'][curve_idx]['sharp']]
)
nbhood_sharp_vert = nbhood.vertices[nbhood_sharp_vert_indexes]


nbhood_surface_verts = nbhood.vertices[surface['vert_indices']]



display_sharpness(
    nbhood, plot_meshvert=True, meshvert_psize=imaging.resolution_3d / 2,
#     sharp_vert=camera_pose.camera_to_world(imaging.rays_origins), sharpvert_psize=0.5,
    sharp_vert=nbhood_sharp_vert, sharpvert_psize=0.1,
    samples=nbhood_surface_verts, samples_psize=0.1,
    directions=None
)

In [ ]:
projections, distances, _ = self.flat_annotation(points)

In [ ]:
# compute the TSharpDF
try:
    distances, directions, has_sharp = annotator.annotate(
        nbhood, nbhood_features, camera_pose.camera_to_world(noisy_points))
except DataGenerationException as e:
    eprint_t(str(e))

In [ ]:
display_sharpness(
    nbhood, plot_meshvert=True, meshvert_psize=imaging.resolution_3d / 2,
#     sharp_vert=camera_pose.camera_to_world(imaging.rays_origins), sharpvert_psize=0.5,
    sharp_vert=nbhood_sharp_vert, sharpvert_psize=0.5,
    samples=camera_pose.camera_to_world(noisy_points), samples_psize=0.1,
    samples_distances=distances,
    directions=None
)

In [ ]:
# convert everything to images
ray_indexes = np.where(image.ravel() != 0)[0]
noisy_image = imaging.points_to_image(noisy_points, ray_indexes)
normals = imaging.points_to_image(normals, ray_indexes, assign_channels=[0, 1, 2])
distances = imaging.points_to_image(distances.reshape(-1, 1), ray_indexes, assign_channels=[0])
directions = imaging.points_to_image(directions, ray_indexes, assign_channels=[0, 1, 2])

In [ ]:
display_depth_sharpness(
    depth_images=[camera_to_display(image)],
    sharpness_images=[camera_to_display(distances)],
    axes_size=(16, 16), 
)

In [ ]:
smell_raycasting_background.run(image)

In [ ]:
smell_depth_discontinuity._depth_discontinuity_threshold

In [ ]:
smell_depth_discontinuity.run(image)


In [ ]:
smell_mesh_self_intersections.run(mesh)

In [ ]:
# compute statistics
num_sharp_curves = len([curve for curve in nbhood_features['curves'] if curve['sharp']])
num_surfaces = len(nbhood_features['surfaces'])

patch_info = {
    'image': noisy_image,
    'normals': normals,
    'distances': distances,
    'directions': directions,
    'item_id': item.item_id,
    'orig_vert_indices': mesh_vertex_indexes,
    'orig_face_indexes': mesh_face_indexes,
    'has_sharp': has_sharp,
    'num_sharp_curves': num_sharp_curves,
    'num_surfaces': num_surfaces,
    'camera_pose': camera_pose.camera_to_world_4x4,
    'mesh_scale': mesh_scale
}

In [ ]:
point_patches.append(patch_info)

In [ ]:
# save_depth_maps(point_patches, '/logs/abc_0056_test.hdf5')

In [ ]:
# !ls -lah /logs/abc_0056_test.hdf5